In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("../Data/insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
X = df.drop(['bought_insurance'], axis=1)
y = df.bought_insurance

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

In [9]:
X_train.shape

(22, 2)

In [10]:
y_train.shape

(22,)

In [11]:
X_test.shape

(6, 2)

In [12]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [13]:
X_train_scaled.head()

,age,affordibility
7,0.60,0
17,0.58,1
2,0.47,1
8,0.62,1
19,0.18,1


In [52]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics= ['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5000)

Epoch 1/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step - accuracy: 0.5000 - loss: 0.7338
Epoch 2/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5000 - loss: 0.7333
Epoch 3/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5000 - loss: 0.7329
Epoch 4/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5000 - loss: 0.7325
Epoch 5/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5000 - loss: 0.7321
Epoch 6/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5000 - loss: 0.7317
Epoch 7/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5000 - loss: 0.7313
Epoch 8/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5000 - loss: 0.7309
Epoch 9/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5000 - loss: 0.7305
Epoch 10/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5000 - loss: 0.7301
Epoch 11/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5000 - loss: 0.7297
Epoch 12/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy

In [66]:
model.evaluate(X_test_scaled, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.8333 - loss: 0.3994


[0.3994351923465729, 0.8333333134651184]

In [67]:
X_test_scaled

,age,affordibility
6,0.55,0
0,0.22,1
5,0.56,1
14,0.49,1
21,0.26,0
25,0.54,1


In [68]:
model.predict(X_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


array([[0.56930923],
       [0.35747528],
       [0.7793227 ],
       [0.70707923],
       [0.21462725],
       [0.7600611 ]], dtype=float32)

In [69]:
y_test

6     0
0     0
5     1
14    1
21    0
25    1
Name: bought_insurance, dtype: int64

In [70]:
coef, intercept = model.get_weights()

In [71]:
coef, intercept

(array([[5.435479 ],
        [0.9283357]], dtype=float32),
 array([-2.7104802], dtype=float32))

In [72]:
def sigmoid(x):
    import math
    return 1 / (1 + math.exp(-x))

In [73]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

In [74]:
prediction_function(0.26, 0)

0.21462725135187596

In [75]:
prediction_function(0.56, 1)

0.7793227001855196

## Neural Network From Scratch

In [76]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i, epsilon) for i in y_predicted]
    y_predicted_new = [min(i, 1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new) + (1-y_true)*np.log(1-y_predicted_new))

In [77]:
def sigmoid_numpy(X):
    return 1 / (1 + np.exp(-X))

In [78]:
sigmoid_numpy(np.array([12, 0, 1]))

array([0.99999386, 0.5       , 0.73105858])

In [79]:
def gradient_descent(age, affordibility, y_true, epochs, loss_threshold):
    # w1, w2, bias
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)

    for i in range(epochs):
        weighted_sum = w1*age + w2*affordibility + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        #calculating the loss
        loss = log_loss(y_true, y_predicted)
        #calculating derivatives
        w1d = (1/n) * np.dot(np.transpose(age), (y_predicted - y_true))
        w2d = (1/n) * np.dot(np.transpose(affordibility), (y_predicted - y_true))
        bias_d = np.mean(y_predicted - y_true)
        #adjusting weights
        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d
        #printing updates
        print(f"Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}")
        
        if loss<=loss_threshold:
            break

    return w1, w2, bias

In [80]:
gradient_descent(X_train_scaled['age'], X_train_scaled['affordibility'], y_train, 1000, 0.4520)

Epoch:0, w1:0.9785158817880981, w2:0.930803018007831, bias:-0.11655161222741105, loss:0.7337633430835588
Epoch:1, w1:0.9630976857496139, w2:0.8726596731306097, bias:-0.21724461019480207, loss:0.698797197338134
Epoch:2, w1:0.9532581013619037, w2:0.8248581333635253, bias:-0.30343286162070227, loss:0.6734055756220733
Epoch:3, w1:0.9483740263551856, w2:0.7863532578161072, bias:-0.376785508440377, loss:0.655371090708311
Epoch:4, w1:0.9477699804852135, w2:0.7559414368753964, bias:-0.439076563486249, loss:0.6427338618045604
Epoch:5, w1:0.9507825053955263, w2:0.7324013015526463, bias:-0.49202677023149366, loss:0.6339125926931728
Epoch:6, w1:0.9568006342617452, w2:0.7145860119101722, bias:-0.5372071856619928, loss:0.6277141906393874
Epoch:7, w1:0.9652857547946362, w2:0.7014720392582826, bias:-0.575994332978751, loss:0.6232814375459
Epoch:8, w1:0.9757770676868515, w2:0.6921766812006341, bias:-0.6095605816307724, loss:0.6200198136399858
Epoch:9, w1:0.9878884607628893, w2:0.6859563946031104, bias:

(np.float64(5.443768295024217),
 np.float64(0.9502568549341589),
 np.float64(-2.7328500621195695))

In [81]:
coef, intercept

(array([[5.435479 ],
        [0.9283357]], dtype=float32),
 array([-2.7104802], dtype=float32))

In [86]:
def prediction_function2(age, affordibility):
    weighted_sum = 5.435479*age + 0.9283357*affordibility - 2.7104802
    return sigmoid(weighted_sum)

In [87]:
X_test_scaled

,age,affordibility
6,0.55,0
0,0.22,1
5,0.56,1
14,0.49,1
21,0.26,0
25,0.54,1


In [89]:
prediction_function2(0.55, 0)

0.5693091963791684

In [90]:
prediction_function2(0.22, 1)

0.35747527031807413

In [91]:
prediction_function2(0.56, 1)

0.7793226972979673

In [92]:
prediction_function2(0.49, 1)

0.7070791571383196

In [93]:
prediction_function2(0.26, 0)

0.2146272471819012

In [94]:
prediction_function2(0.54, 1)

0.7600610348600858

In [96]:
y_test

6     0
0     0
5     1
14    1
21    0
25    1
Name: bought_insurance, dtype: int64

In [97]:
model.predict(X_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


array([[0.56930923],
       [0.35747528],
       [0.7793227 ],
       [0.70707923],
       [0.21462725],
       [0.7600611 ]], dtype=float32)